In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

from modelbase.ode import Simulator

import sys
sys.path.append("../Code")

# Import model functions
from get_current_model import get_model
from function_residuals import calculate_residuals, calculate_residuals_ePathways, integrator_kwargs
from robustness_fit_parameters import get_fitting_parameter_dict, p_names
import functions_light_absorption as lip

# Monte Carlo analysis

In [ ]:
mcpar = pd.read_csv("../Results/montecarlo_202405112232_params.csv", index_col=0)
mcres = pd.read_csv("../Results/montecarlo_202405131529_results.csv", index_col=0)

In [ ]:
# Summary
mcres_outcomes = pd.DataFrame(index=mcres.index, columns=["success", "failed", "time-out"])

mcres_outcomes["timeout"] = np.isnan(mcres).any(axis=1)
mcres_outcomes["failed"] = np.isinf(mcres).any(axis=1)
mcres_outcomes["success"] = mcres_succ = np.invert(np.logical_or(mcres_outcomes["timeout"], mcres_outcomes["failed"]))

print(f"Full runs: {mcres_outcomes['success'].sum()}")
print(f"Time-outs: {mcres_outcomes['timeout'].sum()}")
print(f"Failed: {mcres_outcomes['failed'].sum()}")

In [ ]:
# Find simulations with improved objective functions
mcres_improved = mcres.copy()
mcres_improved = mcres_improved - mcres_improved.iloc[0,:]

# Find simulations with improvement in all objective functions
mcres_outcomes["total_better"] = (mcres_improved < 0).all(axis=1)

# Find simulations with improvement in all objective functions
mcres_outcomes["similar_pareto"] = (mcres_improved < 0.1).all(axis=1)

In [ ]:
mcres_outcomes["similar_pareto"].sum()

In [ ]:
mcpar_flat = mcpar.copy()

# Evaluate the strings in fluo_influence as their dict values are stored as string
# Not a goor practice
mcpar_flat["fluo_influence"] = mcpar_flat["fluo_influence"].apply(eval)

# Unpack the fluo influence parameters 
_fluo_influence = pd.DataFrame(mcpar_flat["fluo_influence"].to_dict()).T
_fluo_influence.columns = "fluo_influence_" + _fluo_influence.columns

# Append them to the parameters
mcpar_flat = pd.concat([mcpar_flat, _fluo_influence], axis=1)
mcpar_flat = mcpar_flat.drop("fluo_influence", axis=1)

In [ ]:
# Normalise the parameters
mcpar_fnorm = mcpar_flat.copy()
mcpar_fnorm = mcpar_fnorm / mcpar_fnorm.iloc[0,:]


In [ ]:
# Visualise the overall distribution

fig, axes = plt.subplots(1, mcres.shape[1], figsize=(15,5), sharey=True)

for (nam, val), ax in zip(mcres.T.iterrows(), axes.flatten()):
    ax.hist(val[mcres_succ], bins=50)
    ax.set_xlabel(nam)
    ax.axvline(mcres.loc[0,nam], c="k")

fig.suptitle("Distribution of residual functions")
fig.tight_layout()

In [ ]:
# Plot the pairwise comparison of model objectives

fig, axes = plt.subplots(
    mcres.shape[1],
    mcres.shape[1],
    figsize=(10,10),
    sharey=False,
    sharex="col"
)

# Plot the pariwise comparisons as scatter plots
for i, (nam_i, val_i) in enumerate(mcres.T.iterrows()):
    for j, (nam_j, val_j) in enumerate(mcres.T.iterrows()):
        # On the diagonal plot histograms
        if i==j:
            axes[i,j].hist(val_i[mcres_succ], bins=50)
        
        elif i>j:
            axes[i,j].plot(
                val_j,
                val_i,
                marker="o",
                markersize=1,
                linestyle="",
            )

        else:
            axes[i,j].remove()

        if j!=0:
            axes[i,j].get_yaxis().set_ticks([])

    ax.hist(val[mcres_succ], bins=50)
    ax.set_xlabel(nam)
    ax.axvline(mcres.loc[0,nam], c="k")

fig.suptitle("Distribution of objective functions")
fig.tight_layout()

In [ ]:
# calculate_residuals_ePathways(
#     parameter_update={},
#     Schuurmans=None,
#     Benschop_CO2pps=None,
#     Benschop_CO2uMs=None,
#     Benschop2003_low=None,
#     experiment_select_435=None,
#     absorption_coef_PAM435=None,
#     data_points_435=None,
#     strain_params=None,
#     data_points_val=None,
#     point_timing=None,
#     fraction_simulated_points=1,
#     integrator_kwargs=integrator_kwargs,
# )

In [ ]:
fig, ax = plt.subplots(mcres.shape)

In [ ]:
# Visualise with single output
nrows = 7
fig, _axes = plt.subplots(nrows, int(np.ceil((mcpar_flat.shape[1]+1) / nrows)), figsize=(15,15))
axes = _axes.flatten()

plot_type = "scatter"

# Plot the overall distribution
axes[0].hist(mcres[mcres_succ], bins=100, orientation="horizontal")
axes[0].set_xlabel("Frequency")

for i, (nam, vals) in enumerate(mcpar_flat.T.iterrows()):

    if plot_type == "scatter":
        axes[i+1].plot(
            vals[mcres_succ],
            mcres[mcres_succ],
            marker="o",
            ls="",
            markersize=1
        )
        axes[i+1].set_xscale("log")
    elif plot_type == "hex":
        axes[i+1].hexbin(
            np.log10(vals[mcres_succ]),
            mcres[mcres_succ],
        )

    # axes[i+1].plot(
    #     vals[0],
    #     mcres[0],
    #     marker="o",
    #     ls="",
    #     markersize=3,
    #     c="red"
    # )

    axes[i+1].set_xlabel(nam)

# Add the y label
_axes[nrows//2, 0].set_ylabel("Root mean squared residuals [AU]")

# Add a title and improve the layout
fig.suptitle("Residuals under parameter variation")
fig.tight_layout()


plt.show(fig)
plt.close()

## Try linear regression

In [ ]:
 from sklearn.linear_model import LinearRegression

In [ ]:
x = mcpar_fnorm[mcres_succ].to_numpy()
y = mcres[mcres_succ]

model = LinearRegression(fit_intercept=True).fit(x,y)

r_sq = model.score(x, y)
print(f"coefficient of determination: {r_sq}")

print(f"intercept: {model.intercept_}")

print(f"coefficients: {model.coef_}")

In [ ]:
r_sq = model.score(x, y)
print(f"coefficient of determination: {r_sq}")

In [ ]:
fig, ax = plt.subplots()
ax.hexbin(
        np.log10(vals[mcres_succ]),
        mcres[mcres_succ],
)

## Visualise

In [ ]:

mcres[mcres_succ].plot(kind="box")

In [ ]:
mcres.min()

In [ ]:
np.log10(mcpar[mcres_succ].iloc[:,0]).plot(kind='hist')

### Selected mutation

In [ ]:
selected_mutation = 916
par = mcpar.loc[selected_mutation,:].to_dict()
par["fluo_influence"] = eval(par["fluo_influence"])

m,y0 = get_model(verbose=False, check_consistency=False)
m.update_parameters(par)
m.update_parameter("pfd", lip.light_gaussianLED(670, 246))

In [ ]:
s = Simulator(m)
s.initialise(y0)
s.simulate(1e6)

# Local optimisation

In [ ]:
with open("../Results/minimise_202405081435_results.pickle", "rb") as f:
    fit = pickle.load(f)

In [ ]:
fit

In [ ]:
p_optim = get_fitting_parameter_dict(fit.x, p_names)

In [ ]:
test = calculate_residuals(p_optim, save_intermediates=False, return_all=True)

In [ ]:
test[1].shape

In [ ]:
residuals = pd.DataFrame(index=np.arange(7), columns=[1,2,3])

In [ ]:
residuals

In [ ]:
test = pd.Series({1:2, 2:4, 3:6})
test2 = pd.Series({2:"a", 3:"b", 1:"c"})

In [ ]:
from pathlib import Path

In [ ]:
Path("../Results/202404171132_CO2production_wi.dill").is_file()

In [ ]:
test

In [ ]:
",".join(test.values.astype(str))

In [ ]:
residuals.loc[0,:] = test

In [ ]:
residuals.loc[1,:] = test2

In [ ]:
residuals